In [1]:
!pip install -i https://pypi.tuna.tsinghua.edu.cn/simple lightgbm
!pip install -i https://pypi.tuna.tsinghua.edu.cn/simple xgboost
!pip install -i https://pypi.tuna.tsinghua.edu.cn/simple catboost

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import KFold, RepeatedKFold
from scipy import sparse
from catboost import Pool, CatBoostRegressor
from sklearn.model_selection import train_test_split

In [3]:
#显示所有列
pd.set_option('display.max_columns', None)
#显示所有行
pd.set_option('display.max_rows', None)

In [4]:
#导入数据
train_abbr=pd.read_csv("./Dataset/happiness_train_abbr.csv",encoding='ISO-8859-1')
train=pd.read_csv("./Dataset/happiness_train_complete_washed.csv",encoding='ISO-8859-1')
test_abbr=pd.read_csv("./Dataset/happiness_test_abbr.csv",encoding='ISO-8859-1')
test=pd.read_csv("./Dataset/happiness_test_complete_washed.csv",encoding='ISO-8859-1')
test_sub=pd.read_csv("./Dataset/happiness_submit.csv",encoding='ISO-8859-1')

In [5]:
#观察数据大小
print(test.shape)
print(test_sub.shape)
print(train.shape)
#简单查看数据
print(train.head())
#查看数据是否缺失
train.info(verbose=True,null_counts=True)

(2968, 120)
(2968, 2)
(8000, 121)
   id  happiness  survey_type  province  city  county      survey_time  \
0   1          4            1        12    32      59   2015/8/4 14:18   
1   2          4            2        18    52      85  2015/7/21 15:04   
2   3          4            2        29    83     126  2015/7/21 13:24   
3   4          5            2        10    28      51  2015/7/25 17:33   
4   5          4            1         7    18      36   2015/8/10 9:50   

   gender  birth  nationality  religion  religion_freq  edu  edu_status  \
0       1   1959            1         1              1   11           4   
1       1   1992            1         1              1   12           4   
2       2   1967            1         0              3    4           4   
3       2   1943            1         1              1    3           4   
4       2   1994            1         1              1   12           1   

   edu_yr  income  political  join_party  floor_area  property  height

In [6]:
#查看label分布
y_train_=train["happiness"]
# y_train_.value_counts()
print(y_train_.value_counts())
#将-8换成3
y_train_=y_train_.map(lambda x:3 if x==-8 else x)
#让label从0开始
y_train_=y_train_.map(lambda x:x-1)
#train和test连在一起
data = pd.concat([train,test],axis=0,ignore_index=True)
#全部数据大小
print(data.shape)

 4    4818
 5    1410
 3    1159
 2     497
 1     104
-8      12
Name: happiness, dtype: int64
(10968, 121)


In [7]:
#处理时间特征
data['survey_time'] = pd.to_datetime(data['survey_time'],format='%Y-%m-%d %H:%M:%S')
data["weekday"]=data["survey_time"].dt.weekday
data["year"]=data["survey_time"].dt.year
data["quarter"]=data["survey_time"].dt.quarter
data["hour"]=data["survey_time"].dt.hour
data["month"]=data["survey_time"].dt.month

In [8]:
# 把一天的时间分段
def hour_cut(x):
    if 0 <= x < 6:
        return 0
    elif 6 <= x < 8:
        return 1
    elif 8 <= x < 12:
        return 2
    elif 12 <= x < 14:
        return 3
    elif 14 <= x < 18:
        return 4
    elif 18 <= x < 21:
        return 5
    elif 21 <= x < 24:
        return 6

data["hour_cut"] = data["hour"].map(hour_cut)

#做问卷时候的年龄
data["survey_age"]=data["year"]-data["birth"]

#让label从0开始
data["happiness"]=data["happiness"].map(lambda x:x-1)

In [9]:
data["hour_cut"] = data["hour"].map(hour_cut)

#做问卷时候的年龄
data["survey_age"]=data["year"]-data["birth"]

In [11]:
#让label从0开始
data["happiness"]=data["happiness"].map(lambda x:x-1)

# edu_other_data = data["edu_other"]
# print(edu_other_data.value_counts())
# print(11111111111)
# print(edu_other_data.notnull())
# data=data.drop(["edu_other"], axis=1)

#去掉三个缺失值很多的
data=data.drop(["happiness"], axis=1)
data=data.drop(["survey_time"], axis=1)

#是否入党
data["join_party"]=data["join_party"].map(lambda x:0 if pd.isnull(x)  else 1)

In [12]:
data.head()

,id,survey_type,province,city,county,gender,birth,nationality,religion,religion_freq,edu,edu_status,edu_yr,income,political,join_party,floor_area,property,height_cm,weight_jin,health,health_problem,depression,hukou,hukou_loc,media_1,media_2,media_3,media_4,media_5,media_6,leisure_1,leisure_2,leisure_3,leisure_4,leisure_5,leisure_6,leisure_7,leisure_8,leisure_9,leisure_10,leisure_11,leisure_12,socialize,relax,learn,social_neighbor,social_friend,socia_outing,equity,class,class_10_before,class_10_after,class_14,work_exper,work_status,work_yr,work_type,work_manage,insur_1,insur_2,insur_3,insur_4,family_income,family_m,family_status,house,car,invest,son,daughter,minor_child,marital,marital_1st,s_birth,marital_now,s_edu,s_political,s_hukou,s_income,s_work_exper,s_work_status,s_work_type,f_birth,f_edu,f_political,f_work_14,m_birth,m_edu,m_political,m_work_14,status_peer,status_3_before,view,inc_ability,inc_exp,trust_1,trust_2,trust_3,trust_4,trust_5,trust_6,trust_7,trust_8,trust_9,trust_10,trust_11,trust_12,trust_13,neighbor_familiarity,public_service_1,public_service_2,public_service_3,public_service_4,public_service_5,public_service_6,public_service_7,public_service_8,public_service_9,weekday,year,quarter,hour,month,hour_cut,survey_age
0,1,1,12,32,59,1,1959,1,1,1,11,4.0,-2.0,20000,1,1,45.0,2,176,155,3,2,5,5,2,4,2,5,5,4,3,1,4,3,1,2,3,4,1,4,5,4,1,2,4,3,3.0,3.0,2,3,3,3,3,1,1,3.0,30.0,1.0,2.0,1,1,1,2,60000,2,2,1,2,2,1,0,0.0,3,1984.0,1958.0,1984.0,6.0,1.0,5.0,40000.0,5.0,-1.0,-1.0,-2,4,4,1,-2,4,1,1,3,2,4,3,50000.0,4,2,-8,-8,5,3,2,3,4,3,-8,4,1,4,50,60.0,50,50,30.0,30,50,50,50,1,2015,3,14,8,4,56
1,2,2,18,52,85,1,1992,1,1,1,12,4.0,2013.0,20000,1,1,110.0,16,170,110,5,4,3,1,1,2,2,1,3,5,1,2,3,4,3,5,4,3,2,3,4,5,1,2,4,3,6.0,2.0,1,3,6,4,8,5,1,3.0,2.0,1.0,3.0,1,1,1,1,40000,3,4,1,2,2,0,0,0.0,1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1972,3,1,2,1973,3,1,2,1,1,4,2,50000.0,5,4,4,3,5,3,3,3,2,3,3,3,2,3,90,70.0,70,80,85.0,70,90,60,60,1,2015,3,15,7,4,23
2,3,2,29,83,126,2,1967,1,0,3,4,4.0,-2.0,2000,1,1,120.0,6,160,122,4,4,5,1,1,2,2,2,5,1,3,1,4,4,3,5,4,4,2,3,5,5,5,3,4,2,2.0,5.0,2,4,5,4,6,3,2,-1.0,-1.0,-1.0,-1.0,1,1,2,2,8000,3,3,1,2,2,0,2,1.0,3,1990.0,1968.0,1990.0,3.0,1.0,1.0,6000.0,3.0,-1.0,-1.0,-2,1,1,2,-2,1,1,2,2,1,4,2,80000.0,3,3,3,3,4,3,3,3,3,3,-8,3,1,4,90,80.0,75,79,80.0,90,90,90,75,1,2015,3,13,7,3,48
3,4,2,10,28,51,2,1943,1,1,1,3,4.0,1959.0,6420,1,1,78.0,8,163,170,4,4,4,1,2,2,1,1,5,1,1,1,5,2,4,5,4,5,1,1,5,5,5,2,4,4,1.0,6.0,1,4,5,5,7,2,4,-1.0,-1.0,-1.0,-1.0,2,2,2,2,12000,3,3,1,1,2,1,4,0.0,7,1960.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-2,14,1,2,-2,1,1,2,2,1,3,2,10000.0,3,3,4,3,5,3,3,5,4,3,3,3,2,3,100,90.0,70,80,80.0,90,90,80,80,5,2015,3,17,7,4,72
4,5,1,7,18,36,2,1994,1,1,1,12,1.0,2014.0,-1,2,1,70.0,16,165,110,5,5,3,2,3,1,3,4,2,5,5,3,3,3,2,4,4,3,5,2,5,5,1,4,3,4,7.0,5.0,3,2,1,1,1,4,6,-1.0,-1.0,-1.0,-1.0,1,2,2,2,-2,4,3,1,1,2,0,0,0.0,1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1970,6,1,10,1972,4,1,15,3,2,3,-8,200000.0,4,3,3,3,5,5,3,4,3,3,3,3,2,2,50,50.0,50,50,50.0,50,50,50,50,0,2015,3,9,8,2,21


In [13]:
data=data.drop(["id"], axis=1)
X_train_ = data[:train.shape[0]]
X_test_  = data[train.shape[0]:]
target_column = 'happiness'
feature_columns=list(X_test_.columns) 
feature_columns

['survey_type',
 'province',
 'city',
 'county',
 'gender',
 'birth',
 'nationality',
 'religion',
 'religion_freq',
 'edu',
 'edu_status',
 'edu_yr',
 'income',
 'political',
 'join_party',
 'floor_area',
 'property',
 'height_cm',
 'weight_jin',
 'health',
 'health_problem',
 'depression',
 'hukou',
 'hukou_loc',
 'media_1',
 'media_2',
 'media_3',
 'media_4',
 'media_5',
 'media_6',
 'leisure_1',
 'leisure_2',
 'leisure_3',
 'leisure_4',
 'leisure_5',
 'leisure_6',
 'leisure_7',
 'leisure_8',
 'leisure_9',
 'leisure_10',
 'leisure_11',
 'leisure_12',
 'socialize',
 'relax',
 'learn',
 'social_neighbor',
 'social_friend',
 'socia_outing',
 'equity',
 'class',
 'class_10_before',
 'class_10_after',
 'class_14',
 'work_exper',
 'work_status',
 'work_yr',
 'work_type',
 'work_manage',
 'insur_1',
 'insur_2',
 'insur_3',
 'insur_4',
 'family_income',
 'family_m',
 'family_status',
 'house',
 'car',
 'invest',
 'son',
 'daughter',
 'minor_child',
 'marital',
 'marital_1st',
 's_birth',
 '

In [14]:
X_train = np.array(X_train_)
y_train = np.array(y_train_)
X_test  = np.array(X_test_)

In [15]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)

(8000, 125)
(8000,)
(2968, 125)


In [16]:
#自定义评价函数
def myFeval(preds, xgbtrain):
    label = xgbtrain.get_label()
    score = mean_squared_error(label,preds)
    return 'myFeval',score

In [17]:
##### xgb

xgb_params = {"booster":'gbtree','eta': 0.005, 'max_depth': 5, 'subsample': 0.7, 
              'colsample_bytree': 0.8, 'objective': 'reg:linear', 'eval_metric': 'rmse', 'silent': True, 'nthread': 8}
folds = KFold(n_splits=5, shuffle=True, random_state=2018)
oof_xgb = np.zeros(len(train))
predictions_xgb = np.zeros(len(test))

for fold_, (trn_idx, val_idx) in enumerate(folds.split(X_train, y_train)):
    print("fold n°{}".format(fold_+1))
    trn_data = xgb.DMatrix(X_train[trn_idx], y_train[trn_idx])
    val_data = xgb.DMatrix(X_train[val_idx], y_train[val_idx])
    
    watchlist = [(trn_data, 'train'), (val_data, 'valid_data')]
    clf = xgb.train(dtrain=trn_data, num_boost_round=20000, evals=watchlist, early_stopping_rounds=200, verbose_eval=100, params=xgb_params,feval = myFeval)
    oof_xgb[val_idx] = clf.predict(xgb.DMatrix(X_train[val_idx]), ntree_limit=clf.best_ntree_limit)
    predictions_xgb += clf.predict(xgb.DMatrix(X_test), ntree_limit=clf.best_ntree_limit) / folds.n_splits
    
print("CV score: {:<8.8f}".format(mean_squared_error(oof_xgb, y_train_)))

fold n°1
[10:05:47] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.2.0/src/objective/regression_obj.cu:174: reg:linear is now deprecated in favor of reg:squarederror.
[10:05:47] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-rmse:2.49565	valid_data-rmse:2.48132	train-myFeval:6.22829	valid_data-myFeval:6.15696
Multiple eval metrics have been passed: 'valid_data-myFeval' will be used for early stopping.

Will train until valid_data-myFeval hasn't improved in 200 rounds.
[100]	train-rmse:1.61275	valid_data-rmse:1.60309	train-myFeval:2.60096	valid_data-myFeval:2.56988
[200]	train-rmse:1.11482	valid_data-rmse:1.11389	train-myFeval:1.24283	val

In [18]:
##### lgb

param = {'boosting_type': 'gbdt',
         'num_leaves': 20,
         'min_data_in_leaf': 20, 
         'objective':'regression',
         'max_depth':6,
         'learning_rate': 0.01,
         "min_child_samples": 30,
         
         "feature_fraction": 0.8,
         "bagging_freq": 1,
         "bagging_fraction": 0.8 ,
         "bagging_seed": 11,
         "metric": 'mse',
         "lambda_l1": 0.1,
         "verbosity": -1}
folds = KFold(n_splits=5, shuffle=True, random_state=2018)
oof_lgb = np.zeros(len(X_train_))
predictions_lgb = np.zeros(len(X_test_))

for fold_, (trn_idx, val_idx) in enumerate(folds.split(X_train, y_train)):
    print("fold n°{}".format(fold_+1))
   # print(trn_idx)
   # print(".............x_train.........")
   # print(X_train[trn_idx])
  #  print(".............y_train.........")
  #  print(y_train[trn_idx])
    trn_data = lgb.Dataset(X_train[trn_idx], y_train[trn_idx])
    
    val_data = lgb.Dataset(X_train[val_idx], y_train[val_idx])

    num_round = 10000
    clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=200, early_stopping_rounds = 100)
    oof_lgb[val_idx] = clf.predict(X_train[val_idx], num_iteration=clf.best_iteration)
    
    predictions_lgb += clf.predict(X_test, num_iteration=clf.best_iteration) / folds.n_splits

print("CV score: {:<8.8f}".format(mean_squared_error(oof_lgb, y_train_)))

fold n°1
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=30 will be ignored. Current value: min_data_in_leaf=20
Training until validation scores don't improve for 100 rounds
[200]	training's l2: 0.437371	valid_1's l2: 0.46862
[400]	training's l2: 0.372343	valid_1's l2: 0.447065
[600]	training's l2: 0.331823	valid_1's l2: 0.442445
[800]	training's l2: 0.301095	valid_1's l2: 0.440626
Early stopping, best iteration is:
[879]	training's l2: 0.290467	valid_1's l2: 0.440119
fold n°2
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=30 will be ignored. Current value: min_data_in_leaf=20
Training until validation scores don't improve for 100 rounds
[200]	training's l2: 0.43282	valid_1's l2: 0.495337
[400]	training's l2: 0.367528	valid_1's l2: 0.469064
[600]	training's l2: 0.327875	valid_1's l2: 0.46247
[800]	training's l2: 0.298299	valid_1's l2: 0.458965
[1000]	training's l2: 0.273288	valid_1's l2: 0.458053
[1200]	training's l2: 0.251047	valid_1's l2: 0.4578

In [19]:
from catboost import Pool, CatBoostRegressor
# cat_features=[0,2,3,10,11,13,15,16,17,18,19]
from sklearn.model_selection import train_test_split


#X_train_s, X_test_s, y_train_s, y_test_s = train_test_split(X_train_, y_train_, test_size=0.3, random_state=2019)
# train_pool = Pool(X_train_s, y_train_s,cat_features=[0,2,3,10,11,13,15,16,17,18,19])
# val_pool = Pool(X_test_s, y_test_s,cat_features=[0,2,3,10,11,13,15,16,17,18,19])
# test_pool = Pool(X_test_ ,cat_features=[0,2,3,10,11,13,15,16,17,18,19]) 


kfolder = KFold(n_splits=5, shuffle=True, random_state=2019)
oof_cb = np.zeros(len(X_train_))
predictions_cb = np.zeros(len(X_test_))
kfold = kfolder.split(X_train_, y_train_)
fold_=0
#X_train_s, X_test_s, y_train_s, y_test_s = train_test_split(X_train, y_train, test_size=0.3, random_state=2019)
for train_index, vali_index in kfold:
    print("fold n°{}".format(fold_))
    fold_=fold_+1
    k_x_train = X_train[train_index]
    k_y_train = y_train[train_index]
    k_x_vali = X_train[vali_index]
    k_y_vali = y_train[vali_index]
    cb_params = {
         'n_estimators': 100000,
         'loss_function': 'RMSE',
         'eval_metric':'RMSE',
         'learning_rate': 0.05,
         'depth': 5,
         'use_best_model': True,
         'subsample': 0.6,
         'bootstrap_type': 'Bernoulli',
         'reg_lambda': 3
    }
    model_cb = CatBoostRegressor(**cb_params)
    #train the model
    model_cb.fit(k_x_train, k_y_train,eval_set=[(k_x_vali, k_y_vali)],verbose=100,early_stopping_rounds=50)
    oof_cb[vali_index] = model_cb.predict(k_x_vali, ntree_end=model_cb.best_iteration_)
    predictions_cb += model_cb.predict(X_test_, ntree_end=model_cb.best_iteration_) / kfolder.n_splits



print("CV score: {:<8.8f}".format(mean_squared_error(oof_cb, y_train_)))


fold n°0
0:	learn: 0.8179904	test: 0.7823302	best: 0.7823302 (0)	total: 59ms	remaining: 1h 38m 15s
100:	learn: 0.6697672	test: 0.6719697	best: 0.6719618 (98)	total: 560ms	remaining: 9m 13s
200:	learn: 0.6401562	test: 0.6662990	best: 0.6660165 (177)	total: 1.15s	remaining: 9m 28s
300:	learn: 0.6119328	test: 0.6656843	best: 0.6653797 (251)	total: 1.67s	remaining: 9m 14s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.6653796695
bestIteration = 251

Shrink model to first 252 iterations.
fold n°1
0:	learn: 0.8099129	test: 0.8153736	best: 0.8153736 (0)	total: 5.38ms	remaining: 8m 57s
100:	learn: 0.6715617	test: 0.6815596	best: 0.6815596 (100)	total: 784ms	remaining: 12m 55s
200:	learn: 0.6427161	test: 0.6711864	best: 0.6711864 (200)	total: 1.39s	remaining: 11m 29s
300:	learn: 0.6152847	test: 0.6682277	best: 0.6680110 (297)	total: 2.12s	remaining: 11m 42s
400:	learn: 0.5918451	test: 0.6657336	best: 0.6654828 (392)	total: 2.73s	remaining: 11m 18s
Stopped by overfitting det

In [21]:
from sklearn import linear_model
# 将lgb和xgb和ctb的结果进行stacking
train_stack = np.vstack([oof_lgb,oof_xgb,oof_cb]).transpose()
test_stack = np.vstack([predictions_lgb, predictions_xgb,predictions_cb]).transpose()


folds_stack = RepeatedKFold(n_splits=5, n_repeats=2, random_state=2018)
oof_stack = np.zeros(train_stack.shape[0])
predictions = np.zeros(test_stack.shape[0])

for fold_, (trn_idx, val_idx) in enumerate(folds_stack.split(train_stack,y_train)):
    print("fold {}".format(fold_))
    trn_data, trn_y = train_stack[trn_idx], y_train[trn_idx]
    val_data, val_y = train_stack[val_idx], y_train[val_idx]
    
    clf_3 = linear_model.BayesianRidge()
    #clf_3 =linear_model.Ridge()
    clf_3.fit(trn_data, trn_y)
    
    oof_stack[val_idx] = clf_3.predict(val_data)
    predictions += clf_3.predict(test_stack) / 10
    
print("CV score: {:<8.8f}".format(mean_squared_error(oof_stack, y_train_)))

fold 0
fold 1
fold 2
fold 3
fold 4
fold 5
fold 6
fold 7
fold 8
fold 9
CV score: 0.45359712


In [22]:
result=list(predictions)
result=list(map(lambda x: x + 1, result))
test_sub["happiness"]=result
test_sub.to_csv("submit_20201121.csv", index=False)